### Quick data overview

Read the `driver_stats.parquet` file. In this demo, we will treat the records on the parquet files as a stream of continually updats for serveral entities, meaning an entity could have been updated more than once .

In [ ]:
import pandas as pd
df = pd.read_parquet("./data/driver_stats.parquet")
df

Find the unique entities

In [ ]:
df.driver_id.unique()

For each entity, find the latest time stamp it was updated. Those records are the only ones that will be present in the online feature store (Redis)

In [ ]:
frames = []
for driver_id in df.driver_id.unique():
    driver_df = df[ df.driver_id == driver_id]
    driver_df = driver_df[driver_df.event_timestamp == driver_df.event_timestamp.max()]
    frames.append(driver_df)
pd.concat(frames)